# Implementation of the Black-Scholes option price

Under the Black-Scholes (BS) model, the call option price is given by

$$ C(K, S_0, \sigma, T) = e^{-rT}\big(F_0 N(d_1) - K N(d_2) \big)$$

where $r$ is the interest rate, $q$ is the continuous dividend rate, $F_0$ is the forward price,

$$ F_0 = e^{(r-q)T}S_0, $$
and
$$ d_{1,2} = \frac{\log(F_0/K)}{\sigma\sqrt{T}} \pm \sigma\sqrt{T}
$$

We are going to implement the formula using function vs class approaches.

In [1]:
import numpy as np
import scipy.stats as spst
import datetime as dt

## 1. Function approach

In [2]:
texp=2
vol=0.4
spot=120
intr=0
divr=0
strike=150
cp=1

vol_std = vol * np.sqrt(texp)
div_fac = np.exp(-texp*divr)
disc_fac = np.exp(-texp*intr)
forward = spot / disc_fac * div_fac
d1 = np.log(forward/strike)/vol_std + 0.5*vol_std
d2 = d1 - vol_std

price = cp * disc_fac \
    * ( forward * spst.norm.cdf(cp*d1) - strike * spst.norm.cdf(cp*d2) )
print(price)

17.301568584255463


In [3]:
# Black-Scholes option price

def bsm_option_price(strike, spot, vol, texp, intr=0.0, divr=0.0, cp=1):
    """
    Black-Scholes option price
    
    A function version by Prof. Choi.
    
    Parameters:
        strike: strike price
        spot: spot price
        vol: volatility
    
    Return: price    
    """
    vol_std = vol * np.sqrt(texp)
    div_fac = np.exp(-texp*divr)
    disc_fac = np.exp(-texp*intr)
    forward = spot / disc_fac * div_fac
    d1 = np.log(forward/strike)/vol_std + 0.5*vol_std
    d2 = d1 - vol_std

    price = cp * disc_fac \
        * ( forward * spst.norm.cdf(cp*d1) - strike * spst.norm.cdf(cp*d2) )
    return price

In [4]:
bsm_option_price(110, 100, 0.2, 7, 0, 0, 1)

17.255566362164977

In [5]:
bsm_option_price(90, 100, 0.2, 10, 0, 0, -1)

18.891474766003526

### Different ways of using function arguments

In [6]:
### full arguments
c1 = bsm_option_price(105, 100, 0.2, 0.25, 0, 0, 1)

### omit arguments with default vaules
c2 = bsm_option_price(105, 100, 0.2, 0.25)

### you need to pass all required arguments
#c2 = bsm_option_price(105, 100, 0.2)

### use argument names
c3 = bsm_option_price(105, 100, texp=0.25, vol=0.2)

### always put the named args at the end
c3 = bsm_option_price(105, texp=0.25, spot=100, vol=0.2)

c1, c2, c3

(2.064019137898846, 2.064019137898846, 2.064019137898846)

## Disadvantage of function approach:
* The computation in the function has to be repeated every time.
* Some part doesn't have to be repeated: for example, __vol, texp, intr, divr__ don't change often. So   __vol_std, disc_fac, div_fac__ don't have to be computed.
* You feel some needs to organize by binding some data + function(method)

## 2. Class approach
We're going to bind __vol, intr, intd__ into a class

In [7]:
class BSM_Ver1:
    """
    BSM Class version 1
    
    """
    vol, texp, intr, divr = None, None, None, None
    
    def __init__(self, vol, texp, intr=0.0, divr=0.0): # Constructor
        self.vol, self.texp, self.intr, self.divr = vol, texp, intr, divr
    
    def price(self, strike, spot, cp=1):
        # cp = 1 for call, -1 for put
        vol_std = self.vol * np.sqrt(self.texp)
        div_fac = np.exp(-self.texp*self.divr)
        disc_fac = np.exp(-self.texp*self.intr)
        forward = spot / disc_fac * div_fac
        d1 = np.log(forward/strike)/vol_std + 0.5*vol_std
        d2 = d1 - vol_std

        price = cp * disc_fac \
            * ( forward * spst.norm.cdf(cp*d1) - strike * spst.norm.cdf(cp*d2) )
        return price

In [8]:
### define a bsm model with spot, vol and expiry.
bsm1 = BSM_Ver1(vol=0.2, texp=1, intr=0.0)
print( bsm1.__dict__ )

{'vol': 0.2, 'texp': 1, 'intr': 0.0, 'divr': 0.0}


In [9]:
### price options with strike
bsm1.price(102, 100), bsm1.price(strike=102, spot=105)

(7.084494247829895, 9.83051012618256)

In [10]:
### Lets change vol/expiry
bsm1.vol = 0.4

### option price changes
bsm1.price(102, 100), bsm1.price(strike=102, spot=100, cp=-1)

(15.029816377425, 17.029816377425)

In [11]:
class BSM_Ver2:
    vol, texp, intr, divr = None, None, None, None

    def __init__(self, vol, texp, intr=0.0, divr=0.0): # Constructor for this class.
        self.vol, self.texp, self.intr, self.divr = vol, texp, intr, divr
        
        self.vol_std = vol * np.sqrt(texp)
        self.div_fac = np.exp(-texp*divr)
        self.disc_fac = np.exp(-texp*intr)
    
    def price(self, strike, spot, cp=1):
        forward = spot / self.disc_fac * self.div_fac
        d1 = np.log(forward/strike)/self.vol_std + 0.5*self.vol_std
        d2 = d1 - self.vol_std
        price = cp * self.disc_fac \
            * ( forward * spst.norm.cdf(cp*d1) - strike * spst.norm.cdf(cp*d2) )
        return price

In [12]:
bsm2 = BSM_Ver2(vol=0.2, texp=1)
print(bsm2.__dict__)

{'vol': 0.2, 'texp': 1, 'intr': 0.0, 'divr': 0.0, 'vol_std': 0.2, 'div_fac': 1.0, 'disc_fac': 1.0}


In [13]:
bsm2.price(102, 100), bsm2.price(102, 100, cp=-1)

(7.084494247829895, 9.08449424782988)

In [14]:
### But... now things are not that simple
bsm2.vol = 0.4
### option prices are price same after volatility change !@#$
bsm2.price(102, 100), bsm2.price(102, 100, cp=-1)
#print( bsm2.__dict__ )

(7.084494247829895, 9.08449424782988)

In [15]:
print(bsm2.__dict__)

{'vol': 0.4, 'texp': 1, 'intr': 0.0, 'divr': 0.0, 'vol_std': 0.2, 'div_fac': 1.0, 'disc_fac': 1.0}


In [16]:
class BSM_Ver3:
    vol, texp, intr, divr = None, None, None, None

    def __init__(self, vol, texp, intr=0.0, divr=0.0): # Constructor
        #self.vol, self.texp, self.intr, self.divr = vol, texp, intr, divr
        self.setparams(vol, texp, intr, divr)

    def setparams(self, vol=None, texp=None, intr=None, divr=None):
        self.vol = vol if(vol != None) else self.vol
        self.texp = texp if(texp != None) else self.texp
        self.intr = intr if(intr != None) else self.intr
        self.divr = divr if(divr != None) else self.divr
        
        self.vol_std = self.vol * np.sqrt(self.texp)
        self.div_fac = np.exp(-self.texp*self.divr)
        self.disc_fac = np.exp(-self.texp*self.intr)
    
    def price(self, strike, spot, cp=1):
        forward = spot / self.disc_fac * self.div_fac
        d1 = np.log(forward/strike)/self.vol_std + 0.5*self.vol_std
        d2 = d1 - self.vol_std
        price = cp * self.disc_fac \
            * ( forward * spst.norm.cdf(cp*d1) - strike * spst.norm.cdf(cp*d2) )
        return price

In [17]:
bsm3 = BSM_Ver3(vol=0.2, texp=1.0)

bsm3.price(102, 100, 1), bsm3.price(102, 100, -1)

(7.084494247829895, 9.08449424782988)

In [18]:
### not let's try setparams method
bsm3.setparams(vol=0.4)
### option prices are price same after volatility change !@#$
bsm3.price(102, 100, 1), bsm3.price(102, 100, -1)

#print( bsm2.__dict__ )

(15.029816377425, 17.029816377425)

In [19]:
class BSM_Ver4:
    _vol, texp, intr, divr = None, None, None, None

    def __init__(self, vol, texp, intr=0.0, divr=0.0): # Constructor
        self._vol, self._texp, self.intr, self.divr = vol, texp, intr, divr
        
        self.vol_std = vol * np.sqrt(texp)
        self.div_fac = np.exp(-texp*divr)
        self.disc_fac = np.exp(-texp*intr)

    @property    
    def vol(self):
        return self._vol

    @vol.setter
    def vol(self, vol):
        self._vol = vol
        self.vol_std = vol * np.sqrt(self.texp)

    @property    
    def texp(self):
        return self._texp

    @texp.setter
    def texp(self, texp):
        self.vol_std = self._vol * np.sqrt(texp)
        self.div_fac = np.exp(-texp*self.divr)
        self.disc_fac = np.exp(-texp*self.intr)
    
    def price(self, strike, spot, cp=1):
        forward = spot / self.disc_fac * self.div_fac
        d1 = np.log(forward/strike)/self.vol_std + 0.5*self.vol_std
        d2 = d1 - self.vol_std
        price = cp * self.disc_fac \
            * ( forward * spst.norm.cdf(cp*d1) - strike * spst.norm.cdf(cp*d2) )
        return price

In [20]:
bsm4 = BSM_Ver4(vol=0.2, texp=1.0)
print(bsm4.vol_std)
bsm4.price(102, 100, 1), bsm4.price(102, 100, -1)

0.2


(7.084494247829895, 9.08449424782988)

In [21]:
### But... now things are not that simple
bsm4.vol = 0.4
print(bsm4.vol_std)
### option prices are price same after volatility change !@#$
bsm4.price(102, 100), bsm4.price(102, 100, cp=-1)
#print( bsm2.__dict__ )

0.4


(15.029816377425, 17.029816377425)

In [22]:
bsm4.texp = 2
print(bsm4.vol_std)

0.5656854249492381


## 3. The class for option contracts 
* This is to be used with the Black-Scholes __model class__ implemented above

In [23]:
class OptionContract:
    def __init__(self, undl, opt_type, strike, dexp):
        ''' Constructor for this class. '''
        self.undl, self.strike, self.dexp = undl, strike, dexp
        self.opt_type = opt_type
        self.cp = 1 if (opt_type == 'call') else -1

    def toString(self):
        return('{:s} option on {:s} struck at {:0.1f} maturing on {:s}'\
              .format(self.opt_type, self.undl, self.strike, self.dexp.strftime('%Y.%m.%d')))

    def price(self, spot, model):
        return model.priceFromContract(spot, self)
    
class BSM_Model(BSM_Ver4): #Class Inheritance
    def priceFromContract(self, spot, contract):
        texp = (contract.dexp - dt.date.today()).days/365.25
        if(abs(texp - self.texp)>1e-12):
            print('Resetting texp of model to {:0.3f}'.format(texp))
            self.texp = texp
        return self.price(contract.strike, spot, contract.cp)

In [24]:
tc_c105_dec = OptionContract('Tencent', 'call', 105, dexp=dt.date(2024, 12, 25))
tc_p95_dec = OptionContract('Tencent', 'put', 95, dexp=dt.date(2024, 12, 25))

print(tc_c105_dec.toString())
print(tc_p95_dec.toString())

call option on Tencent struck at 105.0 maturing on 2024.12.25
put option on Tencent struck at 95.0 maturing on 2024.12.25


In [25]:
bsm_model = BSM_Model(vol=0.2, texp=1)
print(bsm_model.__dict__)

{'_vol': 0.2, '_texp': 1, 'intr': 0.0, 'divr': 0.0, 'vol_std': 0.2, 'div_fac': 1.0, 'disc_fac': 1.0}


In [26]:
tc_spot = 120
price1 = tc_c105_dec.price(tc_spot, model=bsm_model)
price2 = tc_p95_dec.price(tc_spot, model=bsm_model)
print( price1, price2 )

### Similarly....
price3 = bsm_model.priceFromContract(tc_spot, tc_c105_dec)
price4 = bsm_model.priceFromContract(tc_spot, tc_p95_dec)
print( price3, price4 )

Resetting texp of model to 0.819
Resetting texp of model to 0.819
17.713916986156278 0.8940504602153698
Resetting texp of model to 0.819
Resetting texp of model to 0.819
17.713916986156278 0.8940504602153698


In [27]:
import random
class Choi_Model:
    def __init__(self):
        pass
    def priceFromContract(self, spot, contract):
        return( (spot-contract.strike) + random.random()*10  )

In [28]:
tc_spot = 100
choi_model = Choi_Model()
price1 = tc_c105_dec.price(tc_spot, model=choi_model)
price2 = tc_p95_dec.price(tc_spot, model=choi_model)
print( price1, price2 )

-1.5585616160445772 9.516961173653664


## 4. Implementations in PyFENG package

* PYPI package: https://pypi.org/project/pyfeng/

* Code: https://github.com/PyFE/PyFENG/

* Documentation: https://pyfeng.readthedocs.io/en/latest/

In [29]:
import pyfeng as pf

In [30]:
m = pf.Bsm(sigma=0.2, intr=0.0, divr=0.0)

In [31]:
m.price(102, 100, texp=1)

7.084494247829895

In [32]:
np.arange(80, 121, 10)

array([ 80,  90, 100, 110, 120])

In [33]:
# vectorize
m.price(strike=np.arange(80, 121, 10), spot=100, texp=1)

array([21.18592951, 13.58910812,  7.96556746,  4.29201094,  2.14729881])

In [34]:
sigma = np.array([0.2, 0.5])
sigma = sigma[:, None]
sigma

array([[0.2],
       [0.5]])

In [35]:
m = pf.Bsm(sigma, intr=0.05, divr=0.1) # sigma in axis=0
m.price(strike=[90, 95, 100], spot=100, texp=1.2, cp=[-1,1,1])

array([[ 5.75927238,  7.38869609,  5.52948546],
       [16.812035  , 18.83878533, 17.10541288]])